<a href="https://colab.research.google.com/github/aruntakhur/GEU_ML_Jan_25/blob/main/Aditi_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install groq

In [4]:
groq_key = '***'
from groq import Groq
from google.colab import userdata

In [5]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
client = Groq(api_key=GROQ_API_KEY)

In [6]:
def generate_rationale(prompt: str) -> str:
    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=256,
        top_p=1.0
    )
    return response.choices[0].message.content.strip()


In [7]:
def format_prompt(question: str, style) -> str:
    if style == "cot":
        return f"Q: {question}\nA: Let's think step by step."
    else:
        return f"Q: {question}\nA:"

In [8]:
question = "If a notebook costs 2 dollars and a pen costs 1 dollar, how much do 3 notebooks and 2 pens cost?"
prompt = format_prompt(question, style="cot")

response = generate_rationale(prompt)

print("Prompt:\n", prompt)
print("Groq Output:\n", response)

Prompt:
 Q: If a notebook costs 2 dollars and a pen costs 1 dollar, how much do 3 notebooks and 2 pens cost?
A: Let's think step by step.
Groq Output:
 <think>
First, I need to determine the cost of one notebook and one pen. A notebook costs $2, and a pen costs $1.

Next, I'll calculate the total cost for 3 notebooks by multiplying the cost of one notebook by 3. So, 3 notebooks cost 3 × $2 = $6.

Then, I'll calculate the total cost for 2 pens by multiplying the cost of one pen by 2. Therefore, 2 pens cost 2 × $1 = $2.

Finally, I'll add the cost of the notebooks and pens together to find the total cost. Adding $6 and $2 gives a total of $8.
</think>

**Solution:**

1. **Determine the cost of one notebook and one pen:**
   - Notebook: \$2
   - Pen: \$1

2. **Calculate the total cost for 3 notebooks:**
   \[
   3 \text{ notebooks} \times \$2/\text{notebook} = \$6
   \]

3. **Calculate the total cost for 2 pens:**
   \[
   2 \text{ pens} \times \$1/\text{pen} = \$2
   \]

4.


In [ ]:
from datasets import load_dataset
gsm8k = load_dataset("gsm8k", "main", split="test[:20]")

def evaluate_groq_model(dataset, style="cot"):
    correct = 0
    for item in dataset:
        question = item["question"]
        answer = item["answer"].split('####')[-1].strip()
        prompt = format_prompt(question, style)
        response = generate_rationale(prompt)
        if answer in response:
            correct += 1
    return correct / len(dataset)

accuracy = evaluate_groq_model(gsm8k, style="cot")
print(f"Groq CoT Accuracy: {accuracy:.2%}")